In [ ]:
%matplotlib ipympl
# %matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.special import erf

from IPython.display import HTML

In [ ]:
# function computing a stochastic trajectory of a single particle
# simulation box is defined between 0 and 1 in each direction in arbitrary units
def random_walk(num_steps, dt, D, start = "center"):
    std_dev = np.sqrt(2*D*dt)
    if start == "center": start_pos = np.ones(3)*0.5 # particle starts at (0.5, 0.5, 0.5)
    elif start == "random": start_pos = np.random.random(3) # particle starts at random positions
    steps = np.random.normal(0.0, std_dev, size=(num_steps, 3)) # random steps are drawn from normal distribution
    walk = start_pos + np.cumsum(steps, axis=0) # trajectory is cumulative sum of the random steps
    for frame in walk:
        for i in range(3):
            # if particle exits the box, it is moved to its opposite end (periodic boundary conditions)
            while frame[i] > 1.0:
                frame[i] -= 1.0
            while frame[i] < 0.0:
                frame[i] += 1.0
    return walk

# function updating lines plotted at the animated plot
def update_lines(num, walks, lines, ax):
    for line, walk in zip(lines, walks):
        line.set_data(walk[num-1:num, :2].T)
        line.set_3d_properties(walk[num-1:num, 2])
        ax.set_title('Time={}'.format(num))
    return lines

# main function computing stochastic trajectories of a set of particles
def perform_bd(num_steps, dt, D, start, num_particles):
    return [random_walk(num_steps, dt, D, start) for index in range(num_particles)]

# function visualizing stochastic motion of particles
def vis(walks):
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    num_steps = np.shape(walks)[1]
    lines = [ax.plot([], [], [], 'o')[0] for _ in walks]
    ax.set(xlim3d=(0, 1), xlabel='X')
    ax.set(ylim3d=(0, 1), ylabel='Y')
    ax.set(zlim3d=(0, 1), zlabel='Z')
    ani = animation.FuncAnimation(
        fig, update_lines, num_steps, fargs=(walks, lines, ax), interval=100)
#     ani.save('test.mp4')
#     plt.show()
    html = HTML(ani.to_jshtml())
    display(html)

# function counting particles in a given cuboid defined by three pairs of numbers representing three ranges
def count_in_subbox(traj, subbox):
    num_steps = np.shape(traj)[1]
    num_particles = np.shape(traj)[0]
    count = np.zeros(num_steps, dtype = int)
    for j in range(num_steps):
        for k in range(num_particles):
            if traj[k][j][0] >= subbox[0][0] and traj[k][j][0] <= subbox[0][1]:
                if traj[k][j][1] >= subbox[1][0] and traj[k][j][1] <= subbox[1][1]:
                    if traj[k][j][2] >= subbox[2][0] and traj[k][j][2] <= subbox[2][1]:
                        count[j] += 1
                    else:
                        continue
                else:
                    continue
            else:
                continue
    return count

# function visualizing cuboid and concentration in cuboid in time
def vis_subbox(subbox_definition, particle_count, num_particles):
    fig, ax = plt.subplots()
    DefaultSize = fig.get_size_inches()
    fig.set_size_inches( (DefaultSize[0], DefaultSize[1]) )
    nx=2
    ny=2
    grid = plt.GridSpec(nrows=ny, ncols=nx, wspace=0.5, hspace=0.7)
    axa = plt.subplot(grid[0, 0], projection="3d")
    axb = plt.subplot(grid[0, 1])
    axc = plt.subplot(grid[1, 0])
    axd = plt.subplot(grid[1, 1])
    axc.axis('off')
    axd.axis('off')
    vertices = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1],
    [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]], dtype=int)
    vertices = np.array(subbox_definition)[np.arange(3)[np.newaxis, :].repeat(8, axis=0), vertices]
    edges = np.array([[0, 1], [0, 2], [0, 4], [1, 3], [1, 5], [2, 3], [2, 6],
    [3, 7], [4, 5], [4, 6], [5, 7], [6, 7]], dtype=int)
    _ = axa.set_xlim((0,1))
    _ = axa.set_ylim((0,1))
    _ = axa.set_zlim((0,1))
    _ = axa.set_xlabel('X')
    _ = axa.set_ylabel('Y')
    _ = axa.set_zlabel('Z')
#     _ = axa.plot(*vertices.T, 'o', color = 'red')
    for i, j in edges:
        _ = axa.plot(*vertices[[i, j], :].T, color='r', ls='-')
    _ = axb.set_ylim((-10, num_particles+10))
    _ = axb.set_xlabel('Time')
    _ = axb.set_ylabel('Particle count')
    _ = axb.plot(particle_count)
    
def write_xyz_to_file(traj, filename):
    with open(filename, 'w') as output_file:
        for n_timestep in range(len(traj[0])):
            output_file.write('{}\n'.format(len(traj)))
            output_file.write('{} timestep {}\n'.format(filename, n_timestep+1))
            for n_particle in range(len(traj)):
                output_file.write('X{} {} {} {}\n'.format(n_particle+1, *traj[n_particle][n_timestep]))        

# Task 1
We initialize the simulation by placing particles in the center of simulation box. Then, their trajectories are computed using a following scheme:
$$\boldsymbol{r}\left(t+\Delta t\right) = \boldsymbol{r}\left(t \right) + \sqrt{2 D \Delta t} \boldsymbol{X} ,$$
where $\boldsymbol{r}$ is particle's position, $t$ is time, $\Delta t$ is simulation timestep, $D$ is particle's diffusion coefficient, and $\boldsymbol{X}$ is three-dimensional vector, components of which are standard normal random variables (of zero mean and unit variance).

In [ ]:
plt.close()
np.random.seed(19680801) # seed used to initialize pseudorandom number generator
# same seed means same random numbers!
num_particles = 200 # number of diffusing particles
num_steps = 10000 # number of steps in simulation
dt = 1 # size of the timestep in arbitrary units
D = 0.0000001 # diffusion coefficient in length units squared per time units
# (length is defined relative to box size)
traj = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
plot_every_N_steps = 100 # we cannot visualize every simulation step, because it quickly becomes too memory-consuming
# so in animated figure, we show only timeframes being multiples of N
vis(np.array(traj)[:,::plot_every_N_steps,:])
# we can write xyz file with the trajectories of simulated particles
# write_xyz_to_file(traj, 'task1.xyz')

To compare theoretical expressions and the simulations, we are going to trace how the number of particles in a small cetrally positioned cuboid changes in time.

In [ ]:
plt.close()
subbox_definition = np.array([[0.49, 0.51], [0.49, 0.51], [0.49, 0.51]]) # here we define the cuboid in which we are going to compute the number of particles vs time
count = count_in_subbox(traj, subbox_definition) # here we compute number of particles in the cuboid vs time
vis_subbox(subbox_definition, count, num_particles) # here we visualize the cuboid and the number of particles in it vs time
# last argument is the total number of particles, which allows to set the range of y axis

Theoretically, assuming infinite box (which is reasonable, as in the course of simulation particles do not approach the box edges), the particle density diffusing from a point source at $x_0 = (0,0,0)$ spreads in a following war:
$$n_\mathrm{particles}\left(t;x,y,z\right) = N_\mathrm{particles} \left( 4\pi D t \right)^{-3/2} \exp \left[ -\frac{x^2 + y^2 + z^2}{4Dt} \right] . $$
To obtain the number of particles in a finite cuboid, we intefrate the formula above to obtain:
$$N_\mathrm{particles}\left( t; x_1, x_2, y_1, y_2, z_1, z_2 \right) = \left( \frac{1}{2} \right)^3 N_\mathrm{particles} \cdot \left[ \mathrm{erf}\left( \frac{x_2-0.5}{\sqrt{4Dt}} \right) - \mathrm{erf}\left( \frac{x_1-0.5}{\sqrt{4Dt}} \right] \right) \cdot \left[ \mathrm{erf}\left( \frac{y_2-0.5}{\sqrt{4Dt}} \right) - \mathrm{erf}\left( \frac{y_1-0.5}{\sqrt{4Dt}} \right) \right] \cdot \left[ \mathrm{erf}\left( \frac{z_2-0.5}{\sqrt{4Dt}} \right) - \mathrm{erf}\left( \frac{z_1-0.5}{\sqrt{4Dt}} \right) \right] $$
where $x_1, x_2, y_1, y_2, z_1, z_2$ are limits of ranges defining the cuboid, $N_\mathrm{particles}$ is the total number of particles in whole simulation box, and $\mathrm{erf}$ is the [error function](https://en.wikipedia.org/wiki/Error_function). We are going to compare the results of the expression above and the simulations.

In [ ]:
_ = plt.close()
t = np.arange(1, len(count)+1)
theory = num_particles/8 * ( erf((subbox_definition[0][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[0][0]-0.5)/np.sqrt(4*D*t*dt)) ) \
                         * ( erf((subbox_definition[1][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[1][0]-0.5)/np.sqrt(4*D*t*dt)) ) \
                         * ( erf((subbox_definition[2][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[2][0]-0.5)/np.sqrt(4*D*t*dt)) )
_ = plt.plot(t, theory, '-', color = 'red', label = 'theory')
_ = plt.plot(t, count, ':', color = 'black', label = 'sim.')
_ = plt.legend()
_ = plt.xlabel('Time')
_ = plt.ylabel('Particle count')

# Task 2
We now increase the diffusion coefficient slightly, and monitor the number of particles in a cuboid, which initially does not contain any.

In [ ]:
plt.close()
np.random.seed(93843734)
num_particles = 600 # number of particles is larger now
num_steps = 10000
dt = 1
D = 0.000005 # the diffusion coefficients is now 500 times higher than intially
traj = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
plot_every_N_steps = 100
vis(np.array(traj)[:,::plot_every_N_steps,:])
# write_xyz_to_file(traj, 'task2.xyz')

In [ ]:
plt.close()
subbox_definition = np.array([[0.3, 0.45], [0.3, 0.45], [0.45, 0.55]])
count = count_in_subbox(traj, subbox_definition)
vis_subbox(subbox_definition, count, num_particles)

In [ ]:
_ = plt.close()
t = np.arange(1, len(count)+1)
theory = num_particles/8 * ( erf((subbox_definition[0][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[0][0]-0.5)/np.sqrt(4*D*t*dt)) ) \
                         * ( erf((subbox_definition[1][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[1][0]-0.5)/np.sqrt(4*D*t*dt)) ) \
                         * ( erf((subbox_definition[2][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[2][0]-0.5)/np.sqrt(4*D*t*dt)) )
_ = plt.plot(t, theory, '-', color = 'red', label = 'theory')
_ = plt.plot(t, count, ':', color = 'black', label = 'sim.')
_ = plt.legend()
_ = plt.xlabel('Time')
_ = plt.ylabel('Particle count')

Let's now repeat simulation a few times and average the results to acquire better stattisics, i.e., less noisy line.

In [ ]:
plt.close()
np.random.seed(101)
traj1 = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
np.random.seed(301)
traj2 = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
np.random.seed(501)
traj3 = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
np.random.seed(701)
traj4 = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
np.random.seed(901)
traj5 = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
counts = [ count_in_subbox(t, subbox_definition) for t in [traj, traj1, traj2, traj3, traj4, traj5] ]

In [ ]:
_ = plt.close()
t = np.arange(1, len(count)+1)
theory = num_particles/8 * ( erf((subbox_definition[0][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[0][0]-0.5)/np.sqrt(4*D*t*dt)) ) \
                         * ( erf((subbox_definition[1][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[1][0]-0.5)/np.sqrt(4*D*t*dt)) ) \
                         * ( erf((subbox_definition[2][1]-0.5)/np.sqrt(4*D*t*dt)) - erf((subbox_definition[2][0]-0.5)/np.sqrt(4*D*t*dt)) )
_ = plt.plot(t, theory, '-', color = 'red', label = 'theory')
_ = plt.plot(t, np.mean(counts, axis=0), ':', color = 'black', label = 'sim. (mean)')
_ = plt.legend()
_ = plt.xlabel('Time')
_ = plt.ylabel('Particle count')

# Task 3
Now, we increase the diffusion coefficient, so that the particles reach the boundary and mix well. If particles exits the box from one side, it appears at the opposite side (so-called periodic boundary conditions).

In [ ]:
plt.close()
np.random.seed(4383743)
num_particles = 200
num_steps = 10000
dt = 1
D = 0.0001 # the diffusion coefficients is now 1000 times higher than initially
traj = perform_bd(num_steps=num_steps, dt = dt, D = D, start="center", num_particles=num_particles)
plot_every_N_steps = 100
vis(np.array(traj)[:,::plot_every_N_steps,:])
# write_xyz_to_file(traj, 'task3.xyz')

In [ ]:
plt.close()
subbox_definition = np.array([[0.5, 1.0], [0.0, 0.5], [0.0, 1.0]])
count = count_in_subbox(traj, subbox_definition)
vis_subbox(subbox_definition, count, num_particles)

If the particles are well-mixed, the number of particles in a given cuboid is equal to the ratio of the cuboid's volume to the box volume.

In [ ]:
_ = plt.close()
volume = (subbox_definition[0][1]-subbox_definition[0][0])*(subbox_definition[1][1]-subbox_definition[1][0])*(subbox_definition[2][1]-subbox_definition[2][0])
theory = num_particles * volume * np.ones(len(count))
_ = plt.plot(t, theory, '-', color = 'red', label = 'theory')
_ = plt.plot(t, count, ':', color = 'black', label = 'sim.')
_ = plt.legend()
_ = plt.xlabel('Time')
_ = plt.ylabel('Particle count')